https://colab.research.google.com/drive/1FPTx1RXtBfc4MaTkf7viZZD4U2F9gtKN?usp=sharing#scrollTo=nAPe_RVrCTeO

In [18]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [19]:

def gradf(a, b, c):
    return [-3*(a**2) - 0.5*(a**(-0.5)), 3*(cos(3*b)) + 2.5*(b**1.5), 1/(c**2)]

ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")

#check


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [22]:
h = 0.00001
#using simple derivative
print("using simple gradient")
x = [(f(10.0+h, 20.0, 30.0) - f(10.0, 20.0, 30.0))/(h), (f(10.0, 20.0+ h, 30.0) - f(10.0, 20.0, 30.0))/(h), (f(10.0, 20.0, 30.0+h) - f(10.0, 20.0, 30.0))/(2*h)  ]
print(x)
#using symmetric derivative
print("using symmetric gradient")
y = [(f(10.0+h, 20.0, 30.0) - f(10.0-h, 20.0, 30.0))/(2*h), (f(10.0, 20.0+ h, 30.0) - f(10.0, 20.0 - h, 30.0))/(2*h), (f(10.0, 20.0, 30.0+h) - f(10.0, 20.0, 30.0-h))/(2*h)  ]
print(y)
print("Using grad function")
print(gradf(10.0, 20.0, 30.0))

using simple gradient
[-300.1584138360158, 220.74965637557395, 0.000555553469894221]
using symmetric gradient
[-300.15811386761015, 220.74955880952982, 0.0011111126241303282]
Using grad function
[-300.1581138830084, 220.7495588087335, 0.0011111111111111111]


In [24]:
from math import exp, log

In [28]:


class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
#---------------------------------------(add)
  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    return out

  def __radd__(self, other):
    return self + other
  
  #---------------------------------------(mul and rmul)
  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), 'mul')

    def _backward():
      self.grad += other.data*out.grad
      other.grad += self.data*out.grad
    out._backward = _backward
    return out
  
  def __rmul__(self, other):
    return self * other
#----------------------------------------(sub)
  def __sub__(self, other):
    return self + (-other)
  
  def __neg__(self):
    return self*(-1)
  
  def __rsub__(self, other):
    return self + (-other)
  
  #-----------------------------------------(truediv and pow)
  def __truediv__(self, other):
    return self * (other**-1)
  
  def __pow__(self, other):
    assert isinstance(other, (int, float)), "cannot support beyond int/float"
    out = Value(self.data**other, (self,), 'pow') 

    def _backward():
      self.grad = other * ((self.data) **(other - 1)) * out.grad
    out._backward = _backward
    return out
  # ---------------------------------------(exp)
  def exp(self) -> 'Value':
    x = self.data
    out = Value(exp(x), (self, ), 'exp')

    def _backward():
      self.grad += out.data*out.grad
    out._backward = _backward
    return out
  #------------------------------------------(log)
  def log(self):
    out = Value(log(self.data), (self, ), 'log')

    def _backward():
      self.grad += (1/self.data)*out.grad
    out._backward = _backward
    return out
  # -----------------------------------------
  # re-implement all the other functions needed for the exercises below : ?
  # your code here
  # TODO
  #need to implement:
  # - exp(done)
  # - radd(done)
  # - sub, neg, rsub (done)
  # - div-called truediv (done)
  # - mul, rmul
  # - ah sh#t there is a log!
  #__________________________________________ Backward Pass Graph Computation __________________________________________________#
  def backward(self): # exactly as in video  
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [29]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


#### Yay lol!

In [42]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch
logits = torch.tensor([0.0, 3.0, -2.0, 1.0], requires_grad=True)

probs = logits.softmax(dim=0)

loss = -torch.log(probs[3])

loss.backward()


In [43]:
print("Gradients:", logits.grad)

Gradients: tensor([ 0.0418,  0.8390,  0.0057, -0.8865])


# Yay lol omg